In [1]:
!pip install transformers
!wandb off

W&B offline. Running your script from this directory will only write metadata locally. Use wandb disabled to completely turn off W&B.


In [2]:
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [3]:
# dataset class
class SentimentDataset(Dataset):
  def __init__(self, txt_list, label_list, tokenizer, max_length):
    # define variables
    self.input_ids = []
    self.attn_masks = []
    self.labels = []
    map_label = {-1:0, 0:1, 1:2}

    # iterate through the dataset
    for txt, label in zip(txt_list, label_list):
      # prepare the text
      prep_txt = f'<startoftext>{txt}<sep>{map_label[label]}<endoftext>'
      # tokenize
      encodings_dict = tokenizer(prep_txt, truncation=True, max_length=max_length, padding='max_length')
      # append to list
      self.input_ids.append(torch.tensor(encodings_dict['input_ids']))
      self.attn_masks.append(torch.tensor(encodings_dict['attention_mask']))
      self.labels.append(map_label[label])

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]

In [4]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-valid.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-valid.csv",index=False)

In [5]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-test.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-test.csv",index=False)

In [6]:
tt=pd.read_csv("/kaggle/input/bedata/MahaSent/mahasent_train_new_part1.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("MahaSent.csv")

In [7]:
# Data load function
def load_sentiment_dataset(tokenizer):
  # load dataset and sample all reviews in the dataset
  file_path = 'MahaSent.csv'
  df = pd.read_csv(file_path)
  df = df.sample(len(df), random_state=1)


  # divide into test and train
#   X_train, X_test, y_train, y_test = train_test_split(df['tweet'].tolist(), df['label'].tolist(), shuffle=True, test_size=0.05, random_state=1, stratify=df['label'])
  X_train=df['tweet']
  X_test=df['tweet']
  y_train=df['label']
  y_test=df['label']
  # format into SentimentDataset class
  train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=512)

  # return
  return train_dataset, (X_test, y_test)

In [8]:
model_name = 'l3cube-pune/marathi-gpt'

# seed
torch.manual_seed(42)

# load tokenizer and model
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-gpt", bos_token='<startoftext>', eos_token = '<endoftext>', pad_token='<pad>', sep_token='<sep>')
model = AutoModelForCausalLM.from_pretrained("l3cube-pune/marathi-gpt").cuda()
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50261, 768)

In [9]:
# prepare and load dataset
train_dataset, test_dataset = load_sentiment_dataset(tokenizer)

In [12]:
# TRAIN
# creating training arguments
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir='results', num_train_epochs=1, logging_steps=10, save_strategy="epoch", per_device_train_batch_size=2,
                                  per_device_eval_batch_size=2, warmup_steps=100, weight_decay=0.01, logging_dir='logs')

In [13]:
# start training
from transformers import Trainer
Trainer(model=model,
        args=training_args,
        train_dataset=train_dataset,
        data_collator=lambda data: {
            'input_ids':torch.stack([f[0] for f in data]),
            'attention_mask':torch.stack([f[1] for f in data]),
            'labels':torch.stack([f[0] for f in data])
        }).train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 6056
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 1514
  Number of trainable parameters = 124442880
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
10,6.478000
20,0.615200
30,0.358700
40,0.330900
50,0.304100
60,0.328100
70,0.350700
80,0.275100
90,0.295100
100,0.281000


Saving model checkpoint to results/checkpoint-1514
Configuration saved in results/checkpoint-1514/config.json
Configuration saved in results/checkpoint-1514/generation_config.json
Model weights saved in results/checkpoint-1514/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1514, training_loss=0.3472533145700452, metrics={'train_runtime': 758.9104, 'train_samples_per_second': 7.98, 'train_steps_per_second': 1.995, 'total_flos': 1582384545792000.0, 'train_loss': 0.3472533145700452, 'epoch': 1.0})

In [9]:
!zip -r checkpoint.zip /kaggle/working/results/checkpoint-3029

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  adding: kaggle/working/results/checkpoint-3029/ (stored 0%)
  adding: kaggle/working/results/checkpoint-3029/scheduler.pt (deflated 50%)
  adding: kaggle/working/results/checkpoint-3029/generation_config.json (deflated 24%)
  adding: kaggle/working/results/checkpoint-3029/training_args.bin (deflated 48%)
  adding: kaggle/working/results/checkpoint-3029/optimizer.pt (deflated 9%)
  adding: kaggle/working/results/checkpoint-3029/pytorch_model.bin (deflated 10%)
  adding: kaggle/working/results/checkpoint-3029/rng_state.pth (deflated 32%)
  adding: kaggle/working/results/checkpoint-3029/trainer_state.json (deflated 85%)
  adding: kaggle/working/results/checkpoint-3029/

In [20]:
goed=pd.read_csv('/kaggle/input/bedata/MahaSent/mahasent_train_new_part2.csv')

In [21]:
# goed=goed.loc[:,['marathi_text','three_point']]

In [22]:
# goed.rename(columns = {'marathi_text':'tweet','three_point':'label'}, inplace = True)

In [23]:
tt=goed.iloc[:,:]
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
#   i=re.sub(r"\[NAME]+", "", i)
#   i=re.sub(r"\[नाम]+", "",i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("goed.csv")

In [24]:
# goed.to_csv('goed.csv',index=False)

In [25]:
# Data load function
def load_goed_sentiment_dataset(tokenizer):
  # load dataset and sample all reviews in the dataset
  file_path = '/kaggle/working/goed.csv'
  df = pd.read_csv(file_path)
  df = df.sample(len(df), random_state=1)


  # divide into test and train
#   X_train, X_test, y_train, y_test = train_test_split(df['tweet'].tolist(), df['label'].tolist(), shuffle=True, test_size=0.05, random_state=1, stratify=df['label'])
  X_train=df['tweet']
  X_test=df['tweet']
  y_train=df['label']
  y_test=df['label']
  # format into SentimentDataset class
  train_dataset = SentimentDataset(X_train, y_train, tokenizer, max_length=512)

  # return
  return train_dataset, (X_test, y_test)

In [26]:
# prepare and load dataset
goed_train,goed_test = load_goed_sentiment_dataset(tokenizer)

In [19]:
# # TEST

# # set the model to eval mode
# _ = model.eval()

# # run model inference on all test data
# original_label, predicted_label, original_text, predicted_text = [], [], [], []
# map_label = {-1:0, 0:1, 1:2}

# # iterate over all test data
# from tqdm import tqdm
# import re
# for text, label in tqdm(zip(goed_test[0], goed_test[1])):
#   # create prompt (in compliance with the one used during training)
#   prompt = f'<startoftext>{text}<sep>'
#   # generate tokens
#   generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
#   # perform prediction
#   sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0)
#   # decode the predicted tokens into texts
#   pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
#   # extract the predicted sentiment
#   print(prompt)
#   print(pred_text)
#   try:
#     pred_sentiment = pred_text[-1]
#   except:
#     pred_sentiment="None"

#   # append results
#   original_label.append(map_label[label])
#   predicted_label.append(pred_sentiment)
#   original_text.append(text)
#   predicted_text.append(pred_text)

In [20]:
# import gc
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
original_label

In [27]:
# TEST

# set the model to eval mode
_ = model.eval()

# run model inference on all test data
original_label, predicted_label, original_text, predicted_text = [], [], [], []
map_label = {-1:0, 0:1, 1:2}

# iterate over all test data
from tqdm import tqdm
import re
prompt=[]
final=[]
x=1
for text, label in tqdm(zip(goed_test[0], goed_test[1])):
    prompt.append(f'<startoftext>{text}<sep>')
    original_label.append(map_label[label])
    predicted_label.append(prompt)
    original_text.append(text)
    predicted_text.append(prompt)
#     print(x," ",x%1000)
    if x%100==0:     
        print(x)
#         generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
        tokenizer.padding_side = "left"
        generated = tokenizer(prompt, return_tensors="pt",padding=True).to("cuda")
        # perform prediction.
#         sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0)
        with torch.no_grad():
            sample_outputs = model.generate(**generated, do_sample=False, top_k=50, 
                                      max_length=512, top_p=0.9, temperature=0, 
                                      num_return_sequences=0)
        # decode the predicted tokens into texts
#         pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
        pred_text = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
        # extract the predicted sentiment
        print(pred_text[0])
#         print(tokenizer.batch_decode(sample_outputs, skip_special_tokens=False))

        final.extend(pred_text)
        prompt=[]
    x+=1
# print(final)
tokenizer.padding_side = "left"
generated = tokenizer(prompt, return_tensors="pt",padding=True).to("cuda")
# perform prediction.
#         sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0)
with torch.no_grad():
    sample_outputs = model.generate(**generated, do_sample=False, top_k=50, 
                              max_length=512, top_p=0.9, temperature=0, 
                              num_return_sequences=0)
# decode the predicted tokens into texts
#         pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
pred_text = tokenizer.batch_decode(sample_outputs, skip_special_tokens=True)
# extract the predicted sentiment
final.extend(pred_text)
# final.extend(prompt)
# # append results


0it [00:00, ?it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100


100it [00:24,  4.11it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


नियमानुसार, गावात ५० % आदिवासी आणि बाकी बिगर आदिवासी असल्यास नोकर भरतीत दोघांनाही समान न्याय मिळणार. पण बिगर आदिवासींना न्याय मिळत नाही. राज्यपालांकडे मंजुरीसाठी गेलेले T SC चे ठराव बरेच दिवस प्रलंबित आहेत. याचा सरकार प्राधान्या नं पाठपुरावा करून काय उपाय काढणार? 0
200


200it [00:48,  4.15it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

मी त्यांचा पुतण्या म्हणून का कांचे आले नाव? म्हणून.. माझ्या ले करांना सांगतोय गड्या शेती.. बरा आपला गाव भावुक बा पाचा पो राला सल्ला! हे सगळे खरे वाटले तरीही एक सवाल उरला ठेवीदार ांच्या पैशा वर मग नेमका कुणी मारला डल्ला? ज्यांचे पैसे बुडाले त्या सामान्य माणसाला पण द्या की असाच एखादा सल्ला! 0
5900


5900it [22:40,  4.36it/s]Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


पत्रकारांना बाजार ू म्हणाले आता दुकानदार म्हणाले. लोकशाही चार चौथा स्तंभ ही सुटला नाही 0
6000


6057it [23:03,  4.38it/s]
Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


चिपळूण येथील धरण फुट ुन अनेकजण बेपत्ता झाल्याच्या अत्यंत क्लेश दायक प्रकाराची उच्चस्तरीय चौकशी झाली पाहिजे. विभागाच्या मंत्र्याने नैतिक जबाबदारी स्वीकारून राजीनामा दिला पाहिजे. हाच तो नव महाराष्ट्र का? ज्या करता पुन्हा येईन म्हणता. जाहीर निषेध! 0


In [22]:
len(final)

12501

In [23]:
len(goed_test[0])

12501

In [28]:
y=goed_test[1].reset_index(drop=True)

In [29]:
x=goed_test[0].reset_index(drop=True)

In [30]:
len(test_dataset[0])

6056

In [31]:
kai=pd.DataFrame({'label':y,'original':x,'generated':final})    

In [32]:
kai.to_csv("Approach2_preds.csv")

In [33]:
len(goed_test[0])

12501

In [20]:
preds=pd.read_csv('/kaggle/input/goemotions/Predictions2.csv')

In [21]:
preds

,Unnamed: 0,original_text,predicted_text
0,0,मी आधी केले आहे.,मी आधी केले आहे. 1
1,1,कूल-एड प्या! मोठ्याने हसणे,कूल - एड प्या! मोठ्याने हस णे 1
2,2,तरीही ते लहान नाहीत. हॉलीवूडमध्ये असा कोणताही ...,तरीही ते लहान नाहीत. हॉलीवूड मध्ये असा कोणताही...
3,3,"खूप खूप धन्यवाद, मी अभिप्रायाची प्रशंसा करतो आ...","खूप खूप धन्यवाद, मी अभिप्रा याची प्रशंसा करतो ..."
4,4,नवीन काही घडत नाही. समाजाने कधीच तुटलेल्या माण...,नवीन काही घडत नाही. समाजाने कधीच तुटलेल्या माण...
...,...,...,...
14295,14295,No the whippet sound is WUBWUBWUBWUBWUBWUBWUBW...,No the w hi pp et s ound is W U B W U B W U B ...
14296,14296,"पण अहो, निदान त्यांच्याशी संपर्क साधण्याचा त्र...","पण अहो, निदान त्यांच्याशी संपर्क साधण्याचा त्र..."
14297,14297,कोणीतरी डिस्नेला सांगा. दुष्ट माऊसने त्यांच्या...,कोणीतरी डिस् नेला सांगा. दुष्ट माऊस ने त्यांच्...
14298,14298,न्याय्य वाटते. वर्चस्व पुरेसे मोठे होते. खरं त...,न्याय्य वाटते. वर्चस्व पुरेसे मोठे होते. खरं त...


In [23]:
new_text=list(preds['original_text'])+original_text

In [26]:
len(new_pred_text)

25002

In [25]:
new_pred_text=list(preds['predicted_text'])+final

In [ ]:
# pd.read_csv('')

In [27]:
kai=pd.DataFrame({"original_text":new_text,"predicted_text":new_pred_text})
kai.to_csv("Predictions3.csv")

In [ ]:
df1 = pd.DataFrame({
    'original_text':original_text,
    'original_label':original_label,
    'predicted_label': predicted_label,
    'predicted_text': final
})
df1.to_csv('myCsv.csv', index=False)

In [35]:
ll=[]
gg=[]
for i in range(len(new_pred_text)):
    try:
        ll.append(int(new_pred_text[i].split()[-1]))
        gg.append(new_text[i])
    except:
        print(new_pred_text[i])

होय, मी म्हणायला हवे होते'ते उपरोधिक नाही का'0
' माझी देवी तुरुंगात गेली आणि मला फक्त हा भडक टी - शर्ट मिळाला'0
ते तुमच्यासाठी [ N AM E ] आहे'1
जर त्यांनी असे केले तर ते लु र् कर्स लु र् किन नसतील'0
विस्मय कारक माणूस धन्यवाद ♥ ️ ♥ ️ 2 मार्ग शीर्ष
तो म्हणाला की जे बळी वाचले त्यांनी " एका जाड माणसाला नेम बा जाच्या समोर ढकलले " होय ते खरोखरच'वाईट कल्पना ंची चेष्टा करत आहे'0
माझे बायो खूपच मजेदार आहे... मला मिळाले'जर तुम्ही कॅट फिश असाल तर तुमच्याकडून £ 120. 49 चे कॉल आउट शुल्क आकारले जाईल'1
तू भयंकर वृद्ध झाला आहेस'0
No the w hi pp et s ound is W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W U B W 

In [36]:
gg[:5]

['मी आधी केले आहे.',
 'कूल-एड प्या! मोठ्याने हसणे',
 "तरीही ते लहान नाहीत. हॉलीवूडमध्ये असा कोणताही हॉट 5'4 माणूस नाही ज्यासाठी बहुतेक स्त्रिया हसतील.",
 'खूप खूप धन्यवाद, मी अभिप्रायाची प्रशंसा करतो आणि तो मार्गात पाठवीन.',
 'नवीन काही घडत नाही. समाजाने कधीच तुटलेल्या माणसांबद्दल भान दिले नाही. त्यामुळे ते नैसर्गिक आहे.']

In [34]:
new_text[:5]

['मी आधी केले आहे.',
 'कूल-एड प्या! मोठ्याने हसणे',
 "तरीही ते लहान नाहीत. हॉलीवूडमध्ये असा कोणताही हॉट 5'4 माणूस नाही ज्यासाठी बहुतेक स्त्रिया हसतील.",
 'खूप खूप धन्यवाद, मी अभिप्रायाची प्रशंसा करतो आणि तो मार्गात पाठवीन.',
 'नवीन काही घडत नाही. समाजाने कधीच तुटलेल्या माणसांबद्दल भान दिले नाही. त्यामुळे ते नैसर्गिक आहे.']

In [37]:
kai=pd.DataFrame({"tweet":gg,"label":ll})
kai.to_csv("GoEmotions_pred.csv")

In [ ]:
df1 = pd.DataFrame({
    'original_text':original_text,
    'original_label':original_label,
    'predicted_label': predicted_label,
    'predicted_text': predicted_text
})

In [ ]:
df1.to_csv('myCsv.csv', index=False)

In [ ]:
# prepare and load dataset
valid_dataset = load_goed_sentiment_dataset(tokenizer)

In [27]:
# prompt = '<startoftext>'+goed_test[0][3]+'<sep>'
#   # generate tokens
# generated = tokenizer(f"{prompt}", return_tensors="pt").input_ids.cuda()
#   # perform prediction
# sample_outputs = model.generate(generated, do_sample=False, top_k=50, 
#                                   max_length=512, top_p=0.9, temperature=0, 
#                                   num_return_sequences=0)
#   # decode the predicted tokens into texts
# pred_text = tokenizer.decode(sample_outputs[0], skip_special_tokens=True)
# pred_text

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


## Step 2) Training BERT

In [ ]:
!pip install transformers
!wandb off
import torch
from torch.utils.data import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [ ]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-valid.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-valid.csv",index=False)

In [ ]:
tt=pd.read_csv("/kaggle/input/mahasent/tweets-test.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("tweets-test.csv",index=False)

In [ ]:
tt=pd.read_csv("/kaggle/input/bedata/MahaSent/mahasent_train_new_part1.csv")
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("MahaSent.csv")

In [ ]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)
import pandas as pd
import numpy as np
from numpy import array
import sklearn
!pip install simpletransformers

In [ ]:
import pandas as pd
train_df=pd.read_csv("/kaggle/working/MahaSent.csv")
gen=pd.read_csv("/kaggle/input/bedata/Output_one_dash.csv")
valid_df=pd.read_csv("/kaggle/working/tweets-valid.csv")
test_df=pd.read_csv("/kaggle/working/tweets-test.csv")

In [ ]:
train_df=train_df.iloc[:,2:]
gen=pd.DataFrame({'label':gen['label'],'tweet':gen['generated']})
tweet=[]
label=[]
for i in range(len(gen['tweet'])):
    dic={}
    for j in gen['tweet'][i].split():
        dic[j]=0
    for j in gen['tweet'][i].split():
        dic[j]+=1
#     dic=sorted(dic)
    if dic[max(dic,key=dic.get)]<7:
        tweet.append(gen['tweet'][i])
        label.append(gen['label'][i])
gen=pd.DataFrame({'label':label,'tweet':tweet})
train_df=pd.concat([train_df,gen]).reset_index(drop=True)

In [ ]:
train_df['label'] = train_df['label'].replace(1,2)
test_df['label'] = test_df['label'].replace(1,2)
valid_df['label'] = valid_df['label'].replace(1,2)

train_df['label'] = train_df['label'].replace(0,1)
test_df['label'] = test_df['label'].replace(0,1)
valid_df['label'] = valid_df['label'].replace(0,1)

train_df['label'] = train_df['label'].replace(-1,0)
test_df['label'] = test_df['label'].replace(-1,0)
valid_df['label'] = valid_df['label'].replace(-1,0)

In [ ]:
%mkdir new_data
%cd new_data
train_df.to_csv('tweets-train.csv')
test_df.to_csv('tweets-test.csv')
valid_df.to_csv('tweets-valid.csv')
%cd ..

In [ ]:
!pip install --no-cache-dir transformers sentencepiece
!pip install datasets

In [ ]:
from datasets import load_dataset
data_files={"train":"tweets-train.csv","test":"tweets-test.csv","validation":"tweets-valid.csv"}
dataset = load_dataset('new_data',data_files=data_files)
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/marathi-bert-v2")

def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/marathi-bert-v2", num_labels=3)

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch",num_train_epochs=3)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=compute_metrics,
)
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
import pandas as pd
goe_valid_df=pd.read_csv("/kaggle/input/goemotions/goemotions_subset_val_new.csv")
goe_test_df=pd.read_csv("/kaggle/input/goemotions/goemotions_subset_test_new.csv")
goe_test_df=goe_test_df.iloc[:,3:]
goe_valid_df=goe_valid_df.iloc[:,3:]
goe_test_df=pd.DataFrame({'label':goe_test_df['three_point'],'tweet':goe_test_df['marathi_text']})
goe_valid_df=pd.DataFrame({'label':goe_valid_df['three_point'],'tweet':goe_valid_df['marathi_text']})

In [ ]:
tt=goe_test_df.iloc[:,:]
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=re.sub(r"\[NAME]+", "", i)
  i=re.sub(r"\[नाम]+", "",i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("goed_test.csv")

In [ ]:
tt=goe_valid_df.iloc[:,:]
import re
tts=[]
for i in tt["tweet"]:
  i= re.sub(r"http\S+", "", i)
  i= re.sub(r"pic\S+", "", i)
  i= re.sub(r"#\S+", "", i)
  i= re.sub(r"lokmat\S+", "", i)
  i= re.sub(r"@\S+", "", i)
  i= re.sub(r"fb\S+", "", i)
  i= re.sub(r"twitter\S+", "", i)
  i= re.sub(r"bit\S+", "", i)
  i=re.sub(r"\[NAME]+", "", i)
  i=re.sub(r"\[नाम]+", "",i)
  i=i.strip()
  tts.append(i)
tt["tweet"]=tts
tt.to_csv("goed_valid.csv")

In [ ]:
goe_valid_df=pd.read_csv("goed_valid.csv")
goe_test_df=pd.read_csv("goed_test.csv")

In [ ]:
goe_test_df=goe_test_df.iloc[:,1:]
goe_valid_df=goe_valid_df.iloc[:,1:]
goe_test_df=goe_test_df.drop(4294).reset_index(drop=True)
goe_valid_df=goe_valid_df.drop(1742).reset_index(drop=True)
# train_df['label'] = train_df['label'].replace(1,2)
goe_test_df['label'] = goe_test_df['label'].replace(1,2)
goe_valid_df['label'] = goe_valid_df['label'].replace(1,2)

# train_df['label'] = train_df['label'].replace(0,1)
goe_test_df['label'] = goe_test_df['label'].replace(0,1)
goe_valid_df['label'] = goe_valid_df['label'].replace(0,1)

# train_df['label'] = train_df['label'].replace(-1,0)
goe_test_df['label'] = goe_test_df['label'].replace(-1,0)
goe_valid_df['label'] = goe_valid_df['label'].replace(-1,0)

In [ ]:
%mkdir new_data2
%cd new_data2
# train_df.to_csv('tweets-train.csv')
goe_test_df.to_csv('tweets-test.csv')
goe_valid_df.to_csv('tweets-valid.csv')
%cd ..

In [ ]:
from datasets import load_dataset
data_files={"test":"tweets-test.csv","validation":"tweets-valid.csv"}
dataset2 = load_dataset('new_data2',data_files=data_files)

def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True)

tokenized_datasets2 = dataset2.map(tokenize_function, batched=True)


In [ ]:
predictions = trainer.predict(tokenized_datasets2["validation"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
predictions = trainer.predict(tokenized_datasets2["test"])
preds = np.argmax(predictions.predictions, axis=-1)
metric.compute(predictions=preds, references=predictions.label_ids)